In [6]:
import plotly.express as px
import pandas as pd
import numpy as np
validation_df = pd.read_csv("/tmp/validation_df_epoch99.csv")
df = pd.read_csv("analysis/results_df_find_num_neurons.csv")
df.head()
grouped_df = df.groupby(["activation_name", "architecture_id"]).agg(["mean", "std"]).reset_index()
grouped_df.head()

activation_name architecture_id Unnamed: 0             model_id              \
                                        mean         std     mean         std   
0            gelu            1500       4450  294.689846     4450  294.689846   
1            gelu            1501       4451  294.689846     4451  294.689846   
2            gelu            1502       4452  294.689846     4452  294.689846   
3            gelu            1503       4453  294.689846     4453  294.689846   
4            gelu            1504       4454  294.689846     4454  294.689846   

       loss           num_neurons       
       mean       std        mean  std  
0  0.780226  0.100313           1  0.0  
1  0.737277  0.070896           2  0.0  
2  0.733402  0.062604           3  0.0  
3  0.705097  0.021453           4  0.0  
4  0.706819  0.028991           5  0.0

In [49]:
# px.violin(grouped_df, x="num_neurons", y="loss", orientation="h").update_traces(side="positive", width=4)
# px.scatter_3d(grouped_df, x=("num_neurons", "mean"), y="activation_name", z=("loss", "mean"))
df.groupby("architecture_id").median().head().sort_values("loss")

grouped_df.head().sort_values(( "loss", "mean"))
df.groupby(["activation_name", "architecture_id"]).agg(["mean", "min", "std", "count", lambda x: x.quantile(0.95)]).reset_index().sort_values(by=("loss", "<lambda_0>"))[("model_id","count")].values[0]



20

In [22]:
# validation_df.groupby("architecture_id").mean().sort_values(by="test_overall_acc", ascending=True)
# validation_df.describe()

grouped_df = validation_df.groupby("architecture_id").agg(["mean", "median", "std", "count"])
tmp_df= pd.DataFrame()
tmp_df["num_neurons"] = grouped_df[("num_neurons", "mean")]
tmp_df["activation"] = validation_df.groupby("architecture_id").agg({"activation": "first"})
for k in ["mean", "std", "median"]:
	tmp_df[f"{k}_training_loss"] = grouped_df[("training_loss", k)]
	tmp_df[f"{k}_validation_loss"] = grouped_df[("validation_loss", k)]
	tmp_df[f"{k}_train_overall_acc"] = grouped_df[("train_overall_acc", k)]
	tmp_df[f"{k}_valid_overall_acc"] = grouped_df[("valid_overall_acc", k)]
	tmp_df[f"{k}_test_overall_acc"] = grouped_df[("test_overall_acc", k)]
tmp_df["log_std_validation_loss"] = np.log10(tmp_df["std_validation_loss"])
tmp_df["1-std_validation_loss"] = 1-tmp_df["std_validation_loss"]
tmp_df["median_validation_training_loss_gap"] = tmp_df["median_validation_loss"] - tmp_df["median_training_loss"]
x = "num_neurons"
y = "log_std_validation_loss"
y = "median_validation_loss"
z = "median_test_overall_acc"
color = "median_validation_training_loss_gap"
color="activation"
z = "median_test_overall_acc"
size = "1-std_validation_loss"
# tmp_df.describe()

# validation_df.describe()
px.scatter_3d(tmp_df, x=x, y=y, z=z, color=color, size=size,height=800)

In [12]:
# px.scatter_3d(tmp_df, x="1-std_validation_loss", y="num_neurons", z="mean_test_overall_acc", color="mean_valid_overall_acc", size="mean_validation_loss",height=800)
